## Perinatal Psychiatry Audit 2019:

# Are referrals to South West London Perinatal Psychiatry representative of the local population in terms of socioeconomic and ethnic demographics?

# Heatmap analysis to explore the origins of referrals to perinatal psychiatry 

Notebook by: David K Ryan, Academic Foundation Doctor, St George's Hospital
    
*No patient data is included in this notebook and the data is not publicly available*

Abstract is detailed below. 

This notebook describes the code that was used for data analysis and production of heatmaps. 

______

## Abstract 

### Equality of Access to Perinatal Mental Health Services: Are referrals to Perinatal Psychiatry at St George’s Hospital representative of the Ethnic and Socioeconomic demographics of South-West London?


#### David Ryan, Daniel Murphy, Leena Patel, Ben Nereli

#### Aim and Hypothesis:
The aim of this quality improvement study is to understand whether referrals to the St
George’s Perinatal Mental Health Service (PMHS) are representative of the local population
and explore how ethnicity and socioeconomic status may shape women’s access to PMHS.

#### Background:
Mental ill health is a major cause of mortality and morbidity associated with child-birth and
as a result, there is now a growing interest in improving access to PMHS across the UK.
However, it is unclear whether this investment in PMHS is benefiting segments of the
population that may not traditionally engage with health services, such as women from lower
socioeconomic or minority ethnic groups. As a result, there is concern that this investment
has the potential to exacerbate pre-existing health inequalities – representing a form of
intervention generated inequality.

#### Methods:
Referrals to the PMHS at St George’s Hospital, South-West London during the month of
October 2019 (n = 104) were included. Details including weeks of gestation/post-natal,
referrer and ethnicity data were collected routinely on referral forms and socio-economic data
was accessed through the patient’s postcode using the UK Government English Indices of
Deprivation Resource. We compared the demographic make-up of our referrals with the
wider South-West London census data using the chi-square statistic. Differences in the
socioeconomic status of women according to what professional referred the woman (primary
care, maternity service, health visitor, other) were compared using a one-way ANOVA with
Tukey’s post-hoc test. Heat maps weighted according to deprivation decile were generated to
visualise geographical patterns of referrals.

#### Results:
There was no statistically significant difference between referrals and ethnicity census data
(p=0.8463), although there was a trend that a disproportionately large percentage of White
British women were referred to our service (58.33% of referrals vs. background population
45.60%). Similarly, there was no statistically significant difference between deprivation
quintile of our referrals and the local population (p=0.4734). There was a trend that women
referred by health visitors were from lower socioeconomic groups compared to referrals from
Primary Care Providers (p=0.089). Heat maps visually suggest that our referrals follow local
population densities and geographical proximity to St George’s Hospital.

#### Conclusions:
These findings suggest that there is generally good equality of access to PMHS but that there
also exists a socio-economic bias to how women are referred. It is important that referrers and
service providers appreciate these biases in access to secondary care and are cognisant of
these in clinical decision-making.

________

In [ ]:
#import libraries 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
#import
#data = pd.read_csv()

## Processing of LSOA data 

LSOA: Lower Layer Super Output Areas 

In [ ]:
#Simplify the LSOA column
data['LSOA Name'] = data['LSOA Name'].str.replace(r'(^.*Wandsworth.*$)', 'Wandsworth')
data['LSOA Name'] = data['LSOA Name'].str.replace(r'(^.*Sutton.*$)', 'Sutton')
data['LSOA Name'] = data['LSOA Name'].str.replace(r'(^.*Merton.*$)', 'Merton')
data['LSOA Name'] = data['LSOA Name'].str.replace(r'(^.*Richmond.*$)', 'Richmond')
data['LSOA Name'] = data['LSOA Name'].str.replace(r'(^.*Kingston.*$)', 'Kingston')

## How many referrals were received from each of the different boroughs in SWL? 


In [ ]:
#Total number in the dataset 
data.shape

In [ ]:
data['LSOA Name'].value_counts(dropna=True)

In [ ]:
data['LSOA Name'].value_counts(dropna=True).plot(kind='bar');
plt.title('Referrals to Perinatal Psychiatry in October 2019')
#plt.savefig('borough_referrals.png')

## Who was referred to our clinic? 

#### Status of Pregnancy: 



In [ ]:
status_df = pd.DataFrame(data.Status.value_counts())
status_df.columns = ['Number of Women']
status_df.rename({0.0:'Pregnant', 1.0: 'Post-natal', 2.0: 'Pre-conception', -999: 'Missing'}, inplace=True)
status_df['%']= status_df.apply(lambda row: round((row['Number of Women']/len(data))*100), axis = 1) 
status_df.

In [ ]:
from pandas.plotting import table 

In [ ]:
status_df["Number of Women"].plot(kind="bar", title="Types of Referrals to Perinatal Psychiatry");
plt.savefig('referral_type.png')

#### Stage of Pregnancy:

In [ ]:
stage_df = pd.DataFrame(data[['Status', 'Weeks']])
stage_df.rename({ -999: np.NaN}, inplace=True)

In [ ]:
data.groupby('Status')["Weeks"].agg(['count', 'mean', 'min', 'max'])
stage_df = pd.DataFrame(data.groupby('Status')["Weeks"].agg(['count', 'mean', 'min', 'max']))

In [ ]:
stage_df.columns = ['Number of Referrals', 'Mean Weeks of Gestation or Mean Weeks Post-natal', 'Min Weeks', 'Max Weeks']
stage_df.rename({0.0:'Pregnant', 1.0: 'Post-natal', 2.0: 'Pre-conception', -999: 'Missing'}, inplace=True)

In [ ]:
#bar chart to show mean weeks of gestation/post-natal at referral 
stage_df["Mean Weeks of Gestation or Mean Weeks Post-natal"].plot(kind="bar", title="Referrals to Perinatal Psychiatry");

In [ ]:
#boolean masking to create new dataframes based on conditions
m_pregnant = data['Status'] == 0.0
m_pregnant = data.loc[m_pregnant, :].copy()
m_postnatal = data['Status'] == 1.0
m_postnatal = data.loc[m_postnatal, :].copy()
m_pregnant.dropna(inplace=True)
m_postnatal.dropna(inplace=True)

In [ ]:
m_pregnant['Weeks'].isnull().sum()
m_pregnant['Weeks'].plot(kind='hist')
plt.xlabel('weeks of gestation')
plt.title('What gestation were women referred to perinatal services?');
plt.savefig('gestation_pregnant.png')

In [ ]:
#Distribution plot for weeks of gestation referred to service 
sns.distplot(m_pregnant['Weeks']).set_title('Gestation of Pregnant Women Referred to Perinatal Team');

In [ ]:
#histogram 
m_postnatal['Weeks'].hist(bins=26)

## Who are the women who are being referred early? 


In [ ]:
#boolean masking to create a dataframe for women referred prior to 15 weeks of gestation
m_pregnant_early = m_pregnant.loc[m_pregnant['Weeks']<15, :].copy()
m_pregnant_early['Index of Multiple Deprivation Quintile'].value_counts().sort_index()

In [ ]:
m_pregnant_late = m_pregnant.loc[m_pregnant['Weeks']>15, :].copy()
m_pregnant_late['Index of Multiple Deprivation Quintile'].value_counts().sort_index()

In [ ]:
sns.distplot(m_postnatal['Weeks']).set_title('Gestation of Post-natal Women Referred to Perinatal Team');

In [ ]:
m_pregnant['Weeks'].hist(bins=10)
plt.suptitle("Weeks of gestation for referrals of Pregnant Women");

In [ ]:
m_postnatal['Weeks'].hist(grid=False)
plt.xlabel('Weeks post-natal')
plt.ylabel('Frequency')
plt.suptitle("Weeks of gestation for referrals of Post-natal Women");
plt.savefig('gestation_postnatal.png')

## Who referred women to the perinatal team? 

#### Pregnant women: 



In [ ]:
refer_preg = pd.DataFrame(m_pregnant['Referrer'].value_counts())
refer_preg.columns = ['Count']
refer_preg.rename({0.0:'Primary Care', 1.0: 'Maternity Service', 2.0: 'Health Visitor', 3.0: 'Mental Health Secondary Care'}, inplace=True)
refer_preg.plot(kind='bar');

#### Post-natal women:

In [ ]:
refer_postnatal = pd.DataFrame(m_postnatal['Referrer'].value_counts())
refer_postnatal.columns = ['Count']
refer_postnatal.rename({0.0:'Primary Care', 1.0: 'Maternity Service', 2.0: 'Health Visitor', 3.0: 'Mental Health Secondary Care'}, inplace=True)
refer_postnatal.plot(kind='bar');


#### Ethnicity: 

In [ ]:
ethnicity = pd.DataFrame(data['Ethnicity'].value_counts(dropna=False))
ethnicity.rename({'NaN': "Not Known", 0.0:'White British', np.nan : 'Not Known', 1.0: 'White Other', 2.0: 'South Asian', 3.0: 'Asian Other', 4.0: 'African/Black', 5.0: 'Mixed/Other'}, inplace=True)
ethnicity['%']= ethnicity.apply(lambda row: round((row['Ethnicity']/len(data))*100), axis = 1)

In [ ]:
#Drop participants with missing ethnicity 
ethnicity_no_null = pd.DataFrame(data['Ethnicity'].value_counts(dropna=True))
ethnicity_no_null.rename({'NaN': "Not Known", 0.0:'White British', 1.0: 'White Other', 2.0: 'South Asian', 3.0: 'Asian Other', 4.0: 'African/Black', 5.0: 'Mixed/Other'}, inplace=True)
ethnicity_no_null

In [ ]:
ethnicity['Ethnicity'].plot(kind='barh');
plt.title('Ethnicity')
plt.xlabel('Frequency');
plt.savefig('ethnicity.png')

#### Socioeconomic Status: 2019 English IMD

In [ ]:
IMD_OCT = pd.DataFrame(data['Index of Multiple Deprivation Decile'].value_counts(dropna=True).sort_index())
IMD_OCT.plot(kind='bar');

In [ ]:
x = data['Index of Multiple Deprivation Rank']
x.dropna(inplace=True)
sns.distplot(x);

In [ ]:
plt.hist(x);

In [ ]:
data['Index of Multiple Deprivation Quintile'].value_counts().sort_index()

In [ ]:
data['Index of Multiple Deprivation Quintile'].value_counts().sort_index().plot(kind='bar')
plt.title('Quintile of Deprivation')
plt.ylabel('Frequency')
plt.xlabel('Deprivation from most deprived to least deprived');
plt.savefig('Deprivation.png')

In [ ]:
quintile = pd.DataFrame(data['Quintile'].value_counts().sort_index())

In [ ]:
quintile.plot(kind='bar')

## Education 

In [ ]:
data['Education and Skills Decile'].value_counts(dropna=False)

## Avg Socioeconomic Class of GP vs Health Visitor referral 

In [ ]:
data['Referrer'].replace({0.0:'Primary Care', 1.0: 'Maternity Service', 2.0: 'Health Visitor', 3.0: 'Mental Health Secondary Care'}, inplace=True)

In [ ]:
#plot barchart for socioeconomic status 
data.groupby(by='Referrer')['Index of Multiple Deprivation Quintile'].mean().plot(kind='bar')
plt.title("Average Deprivation Decile According to Health Professional making referral")

plt.figure(figsize=(20,20))


plt.savefig('referrer.png')

In [ ]:
#subgroups for different referral type 
data_0_m = data['Index of Multiple Deprivation Quintile'] == 0 
data_0 = list(data[data_0_m]['Index of Multiple Deprivation Rank'])

data_1_m = data['Referrer'] == 1 
data_1 = list(data[data_1_m]['Index of Multiple Deprivation Quintile'])

data_2_m = data['Referrer'] == 2 
data_2 = list(data[data_2_m]['Index of Multiple Deprivation Quintile'])

data_3_m = data['Referrer'] == 3 
data_3 = list(data[data_3_m]['Index of Multiple Deprivation Quintile'])

In [ ]:
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

In [ ]:
#one-way anova to compare mean socioeconomic status between different referrals 
stats.f_oneway(data_0, data_1, data_2, data_3)

In [ ]:
mc = MultiComparison(data['Index of Multiple Deprivation Rank'], data['Referrer'])
result = mc.tukeyhsd()
 
print(result)
print(mc.groupsunique)

## Heatmap Geolocation: Perinatal Caseload using Gmaps Library

Note: latitute and longitude for each postcode was obtained from public online database 

Note: heatmaps not published but used internally to inform planning for perinatal services locally

In [ ]:
caseload_df = pd.read_csv()

In [ ]:
heatmap_caseload_df = caseload_df[['Latitude ', 'Longitude','Index of Multiple Deprivation Decile']]

In [ ]:
heatmap_caseload_df

In [ ]:
#create a geotuple using zip function
heatmap_caseload_df['geo_tuple'] = list(zip(heatmap_caseload_df['Latitude '], heatmap_caseload_df['Longitude']))
heatmap_caseload_df.dropna(inplace=True)

In [ ]:
#create heat maps using gmaps library 
import gmaps
gmaps.configure(api_key=)

gmaps.figure()

#centre map at St George's Hospital 
SGH_coordinates = (51.425168, -0.177016)

#create figure 
fig = gmaps.figure(center=SGH_coordinates, zoom_level=11)

heatmap_layer = gmaps.heatmap_layer(
    heatmap_caseload_df['geo_tuple'], 
    max_intensity = 10,
    dissipating = True, 
    point_radius = 50, 
    opacity=0.5 )

fig.add_layer(heatmap_layer)
fig